In [3]:
with open('23.txt', 'r') as file:
    data = [int(s) for s in file.read().split(',')]


In [9]:
class Halt(Exception):
    pass


class VM:
    def __init__(self, memory, inputs, outputs):
        self.ops = {1: self.add, 
                    2: self.mul, 
                    3: self.input_, 
                    4: self.output, 
                    5: self.jnz,
                    6: self.jz,
                    7: self.lt,
                    8: self.eq,
                    9: self.rbo,
                    99: self.hcf}
        self.memory = memory
        self.pointer = 0
        self.base = 0
        self.inputs = inputs
        self.outputs = outputs

    def run(self):
        while self.step():
            pass
        
    def step(self):
        try:
            self.ops[self.read(self.pointer) % 100]()
        except Halt:
            return False

    def add(self):
        a, b, c = self.decode(3)
        self.write(c, self.read(a) + self.read(b))

    def mul(self):
        a, b, c = self.decode(3)
        self.write(c, self.read(a) * self.read(b))

    def input_(self):
        a, = self.decode(1)
        self.write(a, self.inputs.pop(0) if self.inputs else -1)

    def output(self):
        a, = self.decode(1)
        self.outputs.append(self.read(a))

    def jnz(self):
        a, b = self.decode(2)
        if self.read(a) != 0:
            self.pointer = self.read(b)

    def jz(self):
        a, b = self.decode(2)
        if self.read(a) == 0:
            self.pointer = self.read(b)

    def lt(self):
        a, b, c = self.decode(3)
        self.write(c, 1 if self.read(a) < self.read(b) else 0)

    def eq(self):
        a, b, c = self.decode(3)
        self.write(c, 1 if self.read(a) == self.read(b) else 0)
        
    def rbo(self):
        a, = self.decode(1)
        self.base += self.read(a)

    def hcf(self):
        raise Halt()

    def decode(self, count):
        mode = self.read(self.pointer) // 100
        self.pointer += 1

        params = []
        for i in range(count):
            value = self.pointer
            if mode % 10 == 0:
                value = self.read(value)
            elif mode % 10 == 2:
                value = self.base + self.read(value)
            params.append(value)
            self.pointer += 1
            mode //= 10
        return params
    
    def read(self, offset):
        self.extend(offset)
        return self.memory[offset]
    
    def write(self, offset, value):
        self.extend(offset)
        self.memory[offset] = value
    
    def extend(self, offset):
        if offset >= len(self.memory):
            self.memory.extend([0] * (offset - len(self.memory) + 1))


## Part 1

In [10]:
vms = [VM(data.copy(), inputs=[i], outputs=[]) for i in range(50)]

result = None
while result is None:
    for vm in vms:
        vm.step()
        while len(vm.outputs) >= 3:
            addr, x, y = [vm.outputs.pop(0) for _ in range(3)]
            if addr == 255:
                result = y
            if addr < len(vms):
                vms[addr].inputs.append(x)
                vms[addr].inputs.append(y)

result


26744

## Part 2

In [21]:
vms = [VM(data.copy(), inputs=[i], outputs=[]) for i in range(50)]

old = (0, 0)
nat = (None, None)
none = 0
while True:
    none += 1
    for vm in vms:
        vm.step()
        while len(vm.outputs) >= 3:
            none = 0
            addr, x, y = [vm.outputs.pop(0) for _ in range(3)]
            if addr == 255:
                nat = (x, y)
            if addr < len(vms):
                vms[addr].inputs.append(x)
                vms[addr].inputs.append(y)
    if none == 1000:
        vms[0].inputs.append(nat[0])
        vms[0].inputs.append(nat[1])
        if nat == old:
            break
        old = nat

nat[1]


19498